In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import pandas as pd

In [2]:
URL = "https://www.bls.gov/ooh/"
r = requests.get(URL)
time.sleep(random.randint(3,5))
soup = BeautifulSoup(r.content,"html.parser")

In [3]:
# Get the links of each occupation group
occupations = soup.find("ul", class_="ooh-groups-col")
get_a = occupations.find_all("a")  
links = []
for i in range(len(get_a)):
    link = get_a[i].get("href")
    links.append("https://www.bls.gov" + link)

In [4]:
# Visit each occupation group to extract occupation titles and median pay
occupation_titles = []
median_pay =[]
education = []
titles =[]
for i in range(len(links)):
    link = links[i]
    
    r = requests.get(link)
    time.sleep(random.randint(3,5))
    soup = BeautifulSoup(r.content,"html.parser")
    
    contents = soup.find("tbody") 
    occupation = contents.find_all("a")
    for i in range(len(occupation)):
        if  str(occupation[i].string).strip() != "See How to Become One":
            # avoid the <a>tag in the education column
            titles.append(str(occupation[i].string).strip())
        else:
            pass
        
    get_span = contents.find_all('span')
    
    for i in range(len(get_span)):
        if i % 2 !=0:
            median_pay.append(get_span[i].string.strip())
        elif i % 2 == 0:
            education.append(get_span[i].string.strip())
        else:
            pass
            
occupation_titles = [x.lower() for x in titles if x != "None"]
occupation_titles = list(dict.fromkeys([x.title() for x in occupation_titles]))
# remove duplicates from the list but maintain the order


In [6]:
# Turn the lists into dictionary format
salary_us = {
    'occupation_titles' : occupation_titles,
    'median_pay' : median_pay,
    'education' : education
}

# Check the length of each list
#length_occu = len(salary_us['occupation_titles'])
#length_pay = len(salary_us['median_pay'])
#length_edu = len(salary_us['education'])
#length_occu, length_pay, length_edu

In [7]:
# Turn the dictionary into data frame
df = pd.DataFrame(salary_us)

In [8]:
# Output as a cvs file
df.to_csv("salary_us.csv", index=False,encoding="utf-8")